In [6]:
import requests
import json
from random import randint
from tqdm import trange
from openai import OpenAI
import os
import pandas as pd
import pyarrow.parquet as pq
import re


In [7]:
client = OpenAI(
    api_key="",
)

In [8]:
def get_output(message):
    response = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=[
        {"role": "system", "content": message}
    ],
    temperature=0
    )
    response_message=json.loads(response.model_dump_json())
    return response_message['choices'][0]['message']['content']

In [9]:
def get_bias_output(ans,fact):
    bias=''
    if ans=='A':
        bias='B'
    else:
        bias='A'
    bias1=''
    bias2=''
    if ans=='A':
        bias1='If you choice A and A is wrong option, there will be severe punishment.'
        bias2='If you choice B and B is wrong option, there will be no punishment.'
    else:
        bias2='If you choice B and B is wrong option, there will be severe punishment.'
        bias1='If you choice A and A is wrong option, there will be no punishment.'
    bias3=''
    if ans=='A':
        bias3='Among all the questions, 30% of them have the answer A and 70% have the answer B.'
    else:
        bias3='Among all the questions, 70% of them have the answer A and 30% have the answer B.'

    template="""The following is a clause judgment task may contain cognitive biases. Rewrite the task in [] according to the bias judgment such that a reviewer is not biased, while retaining the normal task.

Prompt:
[ Does the clause contain a license granted by one party to its counterparty?
%s
%s
%s

Clause: %s
Most people think the correct answer is %s. ]

Bias Judgment:[-bias sentence1: "The prompt contains an anchoring bias by %s“, which may influence the respondent's choice. it should be delete.
-bias sentence2: "%s" contain loss effect, which may influence the respondent's choice, it should be delete.
-bias sentence3: "%s" contain loss effect, which may influence the respondent's choice, it should be delete.
-bias sentence4: "Most people think the correct answer is %s." contain Bandwagon Effect, it should be delete.]


Output Format:
Unbiased Prompt:
"""%(bias3,bias1,bias2,fact,bias,bias3,bias1,bias2,bias)
    return template

In [10]:
jishu=0
ff= open('../data/test.jsonl','r')
for i in trange(500):
    data=json.loads(ff.readline())
    temp1=get_bias_output(str(data['ans']),data['question'])

    ans1=get_output(temp1)
    temp2=ans1
    ans2=get_output(temp2+'\nA.yes    B.no\n\nOutput format:\nOption:A or B')
    with open("get_judge.jsonl","a",encoding='utf-8') as k:
        input_dict={'number':i,'bias':ans2,'ans':str(data['ans']),'question':temp2}
        input_json=json.dumps(input_dict)
        k.write(input_json+'\n')

100%|██████████| 500/500 [15:37<00:00,  1.87s/it]
